In [1]:
from medclip import MedCLIPModel, MedCLIPVisionModelViT, MedCLIPVisionModel
from medclip import MedCLIPProcessor
import pandas as pd
from PIL import Image

/home/runai-home/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load patient data
rsna_csv = pd.read_csv("../CXR/datasets/rsna_patients.csv")
male = rsna_csv[rsna_csv["Patient Gender"] == "M"][:500]
female = rsna_csv[rsna_csv["Patient Gender"] == "F"][:500]
subset_df = pd.concat([male, female])
img_dir = "../CXR/datasets/rsna/"  # Make sure path ends with `/`
female

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,rsna
0,00000003_000.png,no findings,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,1
1,00000003_002.png,no findings,2,3,75,F,PA,2048,2500,0.168,0.168,NaN,1
2,00000003_003.png,no findings,3,3,76,F,PA,2698,2991,0.143,0.143,NaN,1
3,00000003_005.png,no findings,5,3,78,F,PA,2686,2991,0.143,0.143,NaN,1
4,00000005_002.png,no findings,2,5,69,F,AP,2500,2048,0.168,0.168,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,00000975_001.png,no findings,1,975,19,F,PA,2048,2500,0.171,0.171,NaN,1
983,00000979_006.png,no findings,6,979,51,F,PA,2318,2322,0.143,0.143,NaN,1
984,00000979_007.png,no findings,7,979,51,F,PA,2670,2677,0.143,0.143,NaN,1
986,00000982_000.png,no findings,0,982,63,F,AP,2500,2048,0.168,0.168,NaN,1


In [4]:
# load MedCLIP-ResNet50
model = MedCLIPModel(vision_cls=MedCLIPVisionModel)
model.from_pretrained()

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


load model weight from: ./pretrained/medclip-resnet


In [6]:
idx = 20
# prepare for the demo image and texts
processor = MedCLIPProcessor()
image = Image.open(img_dir + female["Image Index"].iloc[idx])
inputs = processor(
    text=["lungs remain severely hyperinflated with upper lobe emphysema", 
        "opacity left costophrenic angle is new since prior exam ___ represent some loculated fluid cavitation unlikely"], 
    images=image, 
    return_tensors="pt", 
    padding=True
    )

# pass to MedCLIP model
model = MedCLIPModel(vision_cls=MedCLIPVisionModelViT)
model.from_pretrained()
model.cuda()
outputs = model(**inputs)
print(outputs.keys())

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at microsoft/swin-tiny-patch4-window7-224 were not used when initializing SwinModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SwinModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SwinModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at emilyalsentzer


 Download pretrained model from: https://storage.googleapis.com/pytrial/medclip-vit-pretrained.zip
load model weight from: ./pretrained/medclip-vit
dict_keys(['img_embeds', 'text_embeds', 'logits', 'loss_value', 'logits_per_text'])


In [9]:
outputs['img_embeds'].shape

torch.Size([1, 512])